In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas

In [3]:
import tensorflow as tf

In [4]:
import re

In [5]:
import numpy as np

In [6]:
sess=tf.InteractiveSession()

In [7]:
training_table=pandas.read_table("/storage/malmaud/tmp/train.txt", sep=" ",header=None)

In [8]:
training_table.columns=["file","box","x1","y1","x2","y2"]
training_table.head()

,file,box,x1,y1,x2,y2
0,/storage/malmaud/kinect/hinder1/color/frames/o...,3,1309.333333,816.000000,1498.666667,898.666667
1,/storage/malmaud/kinect/hinder1/color/frames/o...,3,1306.666667,816.000000,1496.000000,901.333333
2,/storage/malmaud/kinect/hinder1/color/frames/o...,3,1306.666667,818.666667,1496.000000,904.000000
3,/storage/malmaud/kinect/hinder1/color/frames/o...,3,1306.666667,821.333333,1496.000000,906.666667
4,/storage/malmaud/kinect/hinder1/color/frames/o...,3,1322.666667,901.333333,1528.000000,1018.666667


In [9]:
batch_size=10

In [10]:
def get_label(filename):
    m = re.match(r".*/(.*?)\d?/color.*", filename)
    return m.groups()[0]

In [11]:
labels=np.unique([get_label(file) for file in training_table['file']])

In [12]:
N_labels=len(labels)

In [13]:
files = np.unique(training_table["file"])

In [14]:
files=files[:194*batch_size]

In [15]:
np.random.shuffle(files)

In [16]:
filename= tf.placeholder(tf.string)

In [17]:
img = tf.image.decode_jpeg(tf.read_file(filename))

In [18]:
X=tf.placeholder(tf.float32, [None, 1080, 1920, 3])

In [19]:
W1=tf.Variable(tf.random_normal([3, 3, 3, 5], stddev=.01))

In [20]:
X1=tf.nn.conv2d(X, W1, [1,1,1,1],'VALID')

In [21]:
X2=tf.nn.max_pool(X1, [1,2,2,1],[1,2,2,1],'VALID')

In [22]:
X2.get_shape()

TensorShape([Dimension(None), Dimension(539), Dimension(959), Dimension(5)])

In [23]:
x=np.zeros([batch_size,1080,1920,3])

In [24]:
y=np.zeros([batch_size, N_labels])

In [25]:
W=tf.Variable(tf.random_normal([539*959*5, N_labels], stddev=.01))

In [26]:
X_flat=tf.reshape(X2,[batch_size, -1])

In [27]:
Y=tf.nn.softmax(tf.matmul(X_flat, W))

In [28]:
Yhat = tf.placeholder(tf.float32, [None, N_labels])

In [29]:
loss = tf.reduce_sum(-Yhat * Y)

In [30]:
gradW = tf.gradients(loss, [W, W1])

In [31]:
alpha=.01

In [33]:
img_id=0
sess.run(tf.initialize_all_variables())
while img_id < len(files):
    y[:] = 0
    for i in range(batch_size):
        x[i, :, :, :] = sess.run(img, feed_dict={filename: files[img_id]})
        label = get_label(files[img_id])
        for j in range(N_labels):
            if labels[j] == label:
                y[i, j] = 1.0
                break
        img_id+=1
        if img_id%100==0:
            print "On image %s/%s" % (img_id, len(files))
    gW, gW1, l = sess.run([gradW[0], gradW[1], loss], feed_dict={X: x, Yhat: y})
    print "Loss is %.5f" % l
    sess.run([W.assign_sub(alpha*gW), W1.assign_sub(alpha*gW1)])
    
    

Loss is -0.00000
Loss is -0.00000
Loss is -0.00004
Loss is -7.95540
Loss is -8.00000
Loss is -8.00000
Loss is -7.00000
Loss is -6.00000
Loss is -9.00000
On image 100/1940
Loss is -7.00000
Loss is -9.00000
Loss is -8.00000
Loss is -8.00000
Loss is -9.00000
Loss is -7.00000
Loss is -7.00000
Loss is -9.00000
Loss is -8.00000
Loss is -8.00000
On image 200/1940
Loss is -9.00000
Loss is -9.00000
Loss is -9.00000
Loss is -8.00000
Loss is -8.00000
Loss is -10.00000
Loss is -9.00000
Loss is -6.00000
Loss is -7.00000
Loss is -10.00000
On image 300/1940
Loss is -9.00000
Loss is -6.00000
Loss is -9.00000
Loss is -8.00000
Loss is -7.00000
Loss is -9.00000
Loss is -10.00000
Loss is -7.00000
Loss is -8.00000
Loss is -8.00000
On image 400/1940
Loss is -8.00000
Loss is -9.00000
Loss is -7.00000
Loss is -7.00000
Loss is -8.00000
Loss is -7.00000
Loss is -8.00000
Loss is -8.00000
Loss is -10.00000
Loss is -9.00000
On image 500/1940
Loss is -6.00000
Loss is -9.00000
Loss is -7.00000
Loss is -6.00000
Loss 

ResourceExhaustedError: OOM when allocating tensor with shape[10,5,539,959]
	 [[Node: gradients/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/gpu:0"](Conv2D, MaxPool, gradients/Reshape_grad/Reshape)]]
	 [[Node: gradients/Conv2D_grad/Conv2DBackpropFilter/_17 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_135_gradients/Conv2D_grad/Conv2DBackpropFilter", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'gradients/MaxPool_grad/MaxPoolGrad', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel